In [31]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16

In [32]:
plt.rc('figure', autolayout = True)
plt.rc('axes', labelweight = 'bold', titleweight = 'bold', titlesize = 18, titlepad = 10, labelsize = 'large')
plt.rc('image', cmap = 'magma')
warnings.filterwarnings("ignore")

In [33]:
def seed_set(seed = 31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC-OPS'] = '1'
seed_set(31415)

In [34]:
train_dir = 'E:/Abhiram/Projects/NexForceTask7/test'
valid_dir = 'E:/Abhiram/Projects/NexForceTask7/valid'

print("Training directory:",train_dir)
print("Validation directory:",valid_dir)

ds_train_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical', 
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = True,
)

ds_valid_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical', 
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = False,
)


Training directory: E:/Abhiram/Projects/NexForceTask7/test
Validation directory: E:/Abhiram/Projects/NexForceTask7/valid
Found 52 files belonging to 8 classes.
Found 52 files belonging to 8 classes.


In [35]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)

In [36]:
pretrained_base = VGG16(weights = 'imagenet',include_top = False, input_shape = (128,128,3))

pretrained_base.trainable = False

In [37]:
#build
model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation = 'softmax')
])
#compile
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

#train
history = model.fit(
    ds_train,validation_data = ds_valid,
    epochs = 30,
    verbose = 1
)


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0962 - loss: 2.8449 - val_accuracy: 0.3846 - val_loss: 2.0132
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.3654 - loss: 2.3703 - val_accuracy: 0.5000 - val_loss: 1.2367
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5769 - loss: 1.5453 - val_accuracy: 0.8654 - val_loss: 0.6212
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.7500 - loss: 0.7671 - val_accuracy: 0.9231 - val_loss: 0.4083
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.7692 - loss: 0.6180 - val_accuracy: 0.9231 - val_loss: 0.2823
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.8269 - loss: 0.6184 - val_accuracy: 0.9231 - val_loss: 0.2376
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.7500 - loss: 0.4910 - val_accuracy: 0.9231 - val_loss: 0.2249
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.8654 - loss: 0.4546 - val_accuracy: 0.9231 - val_loss: 0.1842
Epoch 9/30
1/1 ━

In [38]:
labels = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [39]:
#preprocess image
def preprocess_img(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [128,128])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [40]:
def predict_class(image_path):
    image = preprocess_img(image_path)
    image = tf.expand_dims(image, axis = 0)
    predictions = model.predict(image)

    predicted_index = tf.argmax(predictions[0]).numpy()
    predicted_class = labels[predicted_index]
    return predicted_class

In [48]:
#test
image_path = 'E:/Abhiram/Projects/NexForceTask7/planets/Saturn1.jpg'

planet = predict_class(image_path)
print("The predicted planet is: ", planet)


"""Paths of test images
Mercury: E:/Abhiram/Projects/NexForceTask7/planets/Mercury1.jpg
Venus: E:/Abhiram/Projects/NexForceTask7/planets/Venus1.jpg
Earth: E:/Abhiram/Projects/NexForceTask7/planets/Earth1.jpg
Mars: E:/Abhiram/Projects/NexForceTask7/planets/Mars1.jpg
Jupiter: E:/Abhiram/Projects/NexForceTask7/planets/Jupiter1.jpg
Saturn: E:/Abhiram/Projects/NexForceTask7/planets/Saturn1.jpg
Uranus: E:/Abhiram/Projects/NexForceTask7/planets/Uranus1.jpg
Neptune E:/Abhiram/Projects/NexForceTask7/planets/Neptune1.jpg
"""

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
The predicted planet is:  Saturn


'Paths of test images\nMercury: E:/Abhiram/Projects/NexForceTask7/planets/Mercury1.jpg\nVenus: E:/Abhiram/Projects/NexForceTask7/planets/Venus1.jpg\nEarth: E:/Abhiram/Projects/NexForceTask7/planets/Earth1.jpg\nMars: E:/Abhiram/Projects/NexForceTask7/planets/Mars1.jpg\nJupiter: E:/Abhiram/Projects/NexForceTask7/planets/Jupiter1.jpg\nSaturn: E:/Abhiram/Projects/NexForceTask7/planets/Saturn1.jpg\nUranus: E:/Abhiram/Projects/NexForceTask7/planets/Uranus1.jpg\nNeptune E:/Abhiram/Projects/NexForceTask7/planets/Neptune1.jpg\n'